In [2]:

from transformers import AutoTokenizer,AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer
import torch
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from api.sec_api import key
from sec_api import QueryApi,ExtractorApi
%matplotlib inline

/Users/allen/virtualenvs/CAPSTONE/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#API_KEY = key

tesla =open(r'./Data/tesla.txt').read()

In [4]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

extractorApi = ExtractorApi(API_KEY)

filing_url_10k = "https://www.sec.gov/Archives/edgar/data/1318605/000156459021004599/tsla-10k_20201231.txt"

items = ["1", "1A", "1B", "2", "3", "4", "5", "6", "7", 
           "7A", "8", "9A", "9B", "10", "11", "12", "13", "14"]

filing_url = 'https://www.sec.gov/Archives/edgar/data/1318605/000156459021004599/tsla-10k_20201231.txt'
tsla = ''
for item in items:
  section_text = extractorApi.get_section(filing_url=filing_url, 
                                          section=item, 
                                          return_type="text")
  tsla+=section_text

In [5]:
import re
def text_preprocessing(text):
    """
    - Remove entity mentions (eg. '@united')
    - Correct errors (eg. '&amp;' to '&')
    @param    text (str): a string to be processed.
    @return   text (Str): the processed string.
    """
    # Remove '@name'
    text = re.sub(r'(@.*?)[\s]', ' ', text)

    # Replace '&amp;' with '&'
    text = re.sub(r'&amp;', '&', text)

    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [6]:
tesla = text_preprocessing(tesla)

In [7]:
tokens = tokenizer.encode_plus(
    tesla, add_special_tokens=False,return_tensors='pt'
)

Token indices sequence length is longer than the specified maximum sequence length for this model (122852 > 512). Running this sequence through the model will result in indexing errors


In [28]:
#split toekns in groups of 510
input_id_chunks = tokens['input_ids'][0].split(510)

In [29]:
len(input_id_chunks)

241

In [37]:
start = torch.tensor([101])
end = torch.tensor([102])
chunk_1 = torch.cat((start,input_id_chunks[0],end),0)

for idx,tensor in enumerate(input_id_chunks):
    blank = torch.tensor(tensor[idx])
    _ = torch.cat((start,blank,end),0)
    all_chunks = torch.stack([_,chunk_1],dim=1)


/var/folders/cn/_lgzxbk177j14_b82nbl10br0000gn/T/ipykernel_73844/271250998.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  blank = torch.tensor(tensor[idx])


RuntimeError: zero-dimensional tensor (at position 1) cannot be concatenated

In [33]:
chunk_1

tensor([  101,  8875,  1015,  1012,  2449, 19184,  2057,  2640,  1010,  4503,
         1010,  9922,  1010,  5271,  1998, 10084,  2152,  1011,  2836,  3929,
         3751,  4683,  1998,  2943,  4245,  1998,  5527,  3001,  1010,  1998,
         3749,  2578,  3141,  2000,  2256,  9084,  2943,  3688,  1012,  2057,
         3227,  5271,  2256,  3688,  3495,  2000,  6304,  1010,  2164,  2083,
         2256,  4037,  1998,  7027,  5269,  1012,  2057,  2036,  3613,  2000,
         4982,  2256,  8013,  1011,  5307,  6502,  2083,  1037,  3795,  2897,
         1997,  4316,  2326,  6401,  1010,  4684,  2326, 20202,  1010,  2303,
         7340,  1010,  3565,  7507, 25858,  3703,  1998,  7688, 18649,  2000,
        23306,  1996,  6923,  9886,  1997,  2256,  3688,  1012,  2057, 17902,
         2836,  1010,  8702, 20724,  1998,  1996,  3808,  1997,  2256,  5198,
         1998, 14877,  1999,  1996,  2640,  1998,  9922,  1997,  2256,  3688,
         1998,  2024,  5719,  2000,  4503,  2440,  2969,  1011, 

inputs = tokenizer.encode(tesla,add_special_tokens=True,truncation=False,return_tensors='pt')

In [20]:
len(new)

512

In [21]:
outputs = model(new)

ValueError: not enough values to unpack (expected 2, got 1)

In [18]:
outputs = model(new)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
model.config.id2label
positive = predictions[:, 0].tolist()
negative = predictions[:, 1].tolist()
neutral = predictions[:, 2].tolist()
table = { "Positive":positive, "Negative":negative, "Neutral":neutral}      
df2 = pd.DataFrame(table, columns = ["Positive", "Negative", "Neutral"])

TypeError: forward() takes from 1 to 11 positional arguments but 513 were given

In [22]:
df2

,Positive,Negative,Neutral
0,0.18719,0.008889,0.803921


In [64]:
tokenizer.decode(tsla_token)

'[CLS] item 1. business overview we design, develop, manufacture, sell and lease high - performance fully electric vehicles and energy generation and storage systems, and offer services related to our sustainable energy products. we generally sell our products directly to customers, including through our website and retail locations. we also continue to grow our customer - facing infrastructure through a global network of vehicle service centers, mobile service technicians, body shops, supercharger stations and destination chargers to accelerate the widespread adoption of our products. we emphasize performance, attractive styling and the safety of our users and workforce in the design and manufacture of our products and are continuing to develop full self - driving technology for improved safety. we also strive to lower the cost of ownership for our customers through continuous efforts to reduce manufacturing costs and by offering financial services tailored to our products. our missio

In [72]:
print(tsla_token)

[101, 8875, 1015, 1012, 2449, 19184, 2057, 2640, 1010, 4503, 1010, 9922, 1010, 5271, 1998, 10084, 2152, 1011, 2836, 3929, 3751, 4683, 1998, 2943, 4245, 1998, 5527, 3001, 1010, 1998, 3749, 2578, 3141, 2000, 2256, 9084, 2943, 3688, 1012, 2057, 3227, 5271, 2256, 3688, 3495, 2000, 6304, 1010, 2164, 2083, 2256, 4037, 1998, 7027, 5269, 1012, 2057, 2036, 3613, 2000, 4982, 2256, 8013, 1011, 5307, 6502, 2083, 1037, 3795, 2897, 1997, 4316, 2326, 6401, 1010, 4684, 2326, 20202, 1010, 2303, 7340, 1010, 3565, 7507, 25858, 3703, 1998, 7688, 18649, 2000, 23306, 1996, 6923, 9886, 1997, 2256, 3688, 1012, 2057, 17902, 2836, 1010, 8702, 20724, 1998, 1996, 3808, 1997, 2256, 5198, 1998, 14877, 1999, 1996, 2640, 1998, 9922, 1997, 2256, 3688, 1998, 2024, 5719, 2000, 4503, 2440, 2969, 1011, 4439, 2974, 2005, 5301, 3808, 1012, 2057, 2036, 29453, 2000, 2896, 1996, 3465, 1997, 6095, 2005, 2256, 6304, 2083, 7142, 4073, 2000, 5547, 5814, 5366, 1998, 2011, 5378, 3361, 2578, 21727, 2000, 2256, 3688, 1012, 2256, 3260,

In [ ]:
inputs = tokenizer(df_list, padding = True, truncation = True, return_tensors='pt') #tokenize text to be sent to model


In [95]:
file = open("tesla.txt", "w")
file.write(tsla)
file.close

<function TextIOWrapper.close()>